<a href="https://colab.research.google.com/github/hiferli/LatLongFinder/blob/main/LatLong_FromPINCodes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [126]:
import requests
import json
import time

import pandas as pd
import numpy as np

In [134]:
# Constants
API_KEY="SOME_API_KEY"
COUNTRY="IN"
API_STRUCTURE=f'https://geocode.maps.co/search?postalcode={pincode}&country={COUNTRY}&api_key={API_KEY}'

In [128]:
# Function to get the Latitude, Longitude and Location from PIN Code
def getLocationName(pincode):
    api_url = f'https://geocode.maps.co/search?postalcode={pincode}&country={COUNTRY}&api_key={API_KEY}'

    try:
        response = requests.get(api_url)

        if response.status_code == 200:
            api_data = response.json()

            if api_data:
                location_info = {
                    "lat": api_data[0].get("lat", None),
                    "lon": api_data[0].get("lon", None),
                    "display_name": api_data[0].get("display_name", None),
                }

                # print(location_info)
                return location_info
            else:
                print("Empty API response.")
                return {}
        else:
            print(f"Error: {response.status_code} - {response.text}")
            return {}
    except Exception as e:
        print(f"Exception: {e}")
        return {}
    finally:
        # Introduce a delay of 1 second to comply with the rate limit
        time.sleep(2)

# Example usage:
pincode_to_search = "400009"
# COUNTRY = "India"  # Replace with the actual country value
# API_KEY = "your_api_key"  # Replace with the actual API key

location_info = getLocationName(pincode_to_search)

if location_info:
    print(f"Location information for PINCODE {pincode_to_search}:")
    print(location_info)
else:
    print("Failed to fetch location information.")


Location information for PINCODE 400009:
{'lat': '18.958917496703297', 'lon': '72.83780332857143', 'display_name': 'Zone 1, Mumbai, Maharashtra, 400009, India'}


In [129]:
# Reading Data from CSV File
df = pd.read_csv('/content/Pin_code.csv')

In [130]:
print(df)

       PIN Code
0        400009
1        400010
2        400012
3        400014
4        400016
...         ...
17366    160079
17367    160106
17368    133301
17369    133302
17370    134101

[17371 rows x 1 columns]


In [131]:
# Taking a sample of the dataframe, since the API has a rate limiter with limit of 2 seconds, hence would take a lot of time to parse all the data
sample_df = df.sample(n=5);
sample_df

,PIN Code
17164,110119
11613,587102
14447,631551
16092,721211
1735,624054


In [132]:
# Getting the details using the API
location_info_df = sample_df['PIN Code'].apply(getLocationName).apply(pd.Series)

# Concatenation of the requested data and the PIN Codes
sample_df = pd.concat([sample_df, location_info_df], axis=1)

# Renaming the columns
sample_df = sample_df.rename(columns={0: 'Latitude', 1: 'Longitude', 2: 'Display Name'})

Empty API response.
Empty API response.


<ipython-input-132-848e80f33905>:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  location_info_df = sample_df['PIN Code'].apply(getLocationName).apply(pd.Series)
<ipython-input-132-848e80f33905>:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  location_info_df = sample_df['PIN Code'].apply(getLocationName).apply(pd.Series)


In [133]:
# Note that whenever the required data can't be found, we return an NaN type
sample_df

,PIN Code,lat,lon,display_name
17164,110119,NaN,NaN,NaN
11613,587102,16.17470234285714,75.66067117142858,"Bagalakote taluk, Bagalkote District, Karnatak..."
14447,631551,12.865887366666668,79.60716343333334,"Kancheepuram, Kanchipuram, Tamil Nadu, 631551,..."
16092,721211,22.58586996,87.69048633999999,"Daspur-I, Paschim Medinipur District, West Ben..."
1735,624054,NaN,NaN,NaN
